In [7]:
# --- Modified Imports for Google Colab ---
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from io import StringIO
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import bootstrap
import os
from statsmodels.tsa.seasonal import STL
import statsmodels.formula.api as sm
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor, plot_tree
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# --- Constants ---
OUTPUT_PATH = "/content/drive/MyDrive/output_anxiety_analysis/"
PARTICIPANT_ID_COLUMN = "participant_id"
GROUP_COLUMN = "group"
ANXIETY_PRE_COLUMN = "anxiety_pre"
ANXIETY_POST_COLUMN = "anxiety_post"
BOOTSTRAP_RESAMPLES = 500
N_CLUSTERS = 3
RANDOM_STATE = 42

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Modified Visualization Functions with Neon Colors ---
def create_kde_plot(df, pre_col, post_col, output_path):
    try:
        plt.figure(figsize=(8, 5), facecolor='black')
        ax = plt.gca()
        ax.set_facecolor('black')
        sns.kdeplot(data=df[pre_col], label=f"{pre_col.capitalize()}", color="#00FFFF", fill=True, alpha=0.7)
        sns.kdeplot(data=df[post_col], label=f"{post_col.capitalize()}", color="#FF00FF", fill=True, alpha=0.5)
        plt.title('KDE Plot of Anxiety Levels', color='white')
        plt.xlabel('Anxiety Score', color='white')
        plt.ylabel('Density', color='white')
        plt.legend(facecolor='black', edgecolor='white', labelcolor='white')
        ax.tick_params(colors='white')
        plt.savefig(os.path.join(output_path, 'kde_plot.png'), facecolor='black')
        plt.close()
        return "KDE plot created."
    except Exception as e:
        print(f"Error creating KDE plot: {e}")
        return "Error creating KDE plot."

def create_grouped_kde_plot(df, group_col, pre_col, post_col, output_path):
    try:
        plt.figure(figsize=(12, 6), facecolor='black')
        ax = plt.gca()
        ax.set_facecolor('black')
        colors = {'Group A': '#00FFFF', 'Group B': '#FF00FF', 'Control': '#00FF00'}
        for group in df[group_col].unique():
            subset = df[df[group_col] == group]
            sns.kdeplot(subset[pre_col], label=f'{group} - Pre', color=colors[group], fill=False)
            sns.kdeplot(subset[post_col], label=f'{group} - Post', color=colors[group], linestyle='--', fill=False)
        plt.title('Grouped KDE Plot by Group', color='white')
        plt.xlabel('Anxiety Score', color='white')
        plt.ylabel('Density', color='white')
        plt.legend(facecolor='black', edgecolor='white', labelcolor='white')
        ax.tick_params(colors='white')
        plt.savefig(os.path.join(output_path, 'grouped_kde_plot.png'), facecolor='black')
        plt.close()
        return "Grouped KDE plot created."
    except Exception as e:
        print(f"Error creating grouped KDE plot: {e}")
        return "Error creating grouped KDE plot."

def create_violin_plot(df, group_col, pre_col, post_col, output_path):
    try:
        plt.figure(figsize=(10, 6), facecolor='black')
        ax = plt.gca()
        ax.set_facecolor('black')
        sns.violinplot(x=group_col, y=pre_col, data=df, color="#00FFFF", inner="quart")
        sns.violinplot(x=group_col, y=post_col, data=df, color="#FF00FF", inner="quart", alpha=0.7)
        plt.title('Violin Plot of Anxiety Levels', color='white')
        plt.xlabel('Intervention Group', color='white')
        plt.ylabel('Anxiety Score', color='white')
        ax.tick_params(colors='white')
        plt.savefig(os.path.join(output_path, 'violin_plot.png'), facecolor='black')
        plt.close()
        return "Violin plot created."
    except Exception as e:
        print(f"Error creating violin plot: {e}")
        return "Error creating violin plot."

def create_scatter_plot(df, pre_col, post_col, group_col, output_path):
    try:
        plt.figure(figsize=(8, 6), facecolor='black')
        ax = plt.gca()
        ax.set_facecolor('black')
        sns.scatterplot(x=pre_col, y=post_col, hue=group_col, data=df,
                       palette=['#00FFFF', '#FF00FF', '#00FF00'], s=100)
        plt.title('Pre vs Post Anxiety by Group', color='white')
        plt.xlabel('Pre-Intervention Anxiety', color='white')
        plt.ylabel('Post-Intervention Anxiety', color='white')
        plt.legend(title='Group', facecolor='black', edgecolor='white', labelcolor='white')
        ax.tick_params(colors='white')
        plt.grid(True, linestyle='--', color='white', alpha=0.3)
        plt.savefig(os.path.join(output_path, 'scatter_plot.png'), facecolor='black')
        plt.close()
        return "Scatter plot created."
    except Exception as e:
        print(f"Error creating scatter plot: {e}")
        return "Error creating scatter plot."

def create_decision_tree_plot(df, features, target, output_path):
    try:
        plt.figure(figsize=(12, 8), facecolor='black')
        tree = DecisionTreeRegressor(max_depth=3, random_state=RANDOM_STATE)
        tree.fit(df[features], df[target])
        plot_tree(tree, feature_names=features, filled=True, rounded=True,
                 fontsize=10, impurity=False,
                 class_names=True,
                 precision=2,
                 node_ids=True,
                 proportion=True,
                 ax=plt.gca())
        ax = plt.gca()
        ax.set_facecolor('black')
        for text in ax.get_xticklabels() + ax.get_yticklabels() + ax.texts:
            text.set_color('white')
        plt.title('Decision Tree for Anxiety Prediction', color='white', pad=20)
        plt.savefig(os.path.join(output_path, 'decision_tree.png'), facecolor='black', bbox_inches='tight')
        plt.close()
        return "Decision tree plot created."
    except Exception as e:
        print(f"Error creating decision tree plot: {e}")
        return "Error creating decision tree plot."

# --- Updated Main Script ---
if __name__ == "__main__":
    # Create output directory
    create_output_directory(OUTPUT_PATH)

    # Load synthetic data (unchanged)
    synthetic_data = """
participant_id,group,anxiety_pre,anxiety_post
P001,Group A,4,2
P002,Group A,3,1
P003,Group A,5,3
P004,Group B,6,5
P005,Group B,5,4
P006,Group B,7,6
P007,Control,3,3
P008,Control,4,4
P009,Control,2,2
P010,Control,5,5
"""
    df = load_data_from_synthetic_string(synthetic_data)
    required_columns = [PARTICIPANT_ID_COLUMN, GROUP_COLUMN, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN]
    if not validate_dataframe(df, required_columns):
        exit()

    # Generate statistical summary
    summary_stats = df.describe()
    with open(os.path.join(OUTPUT_PATH, 'summary.txt'), 'w') as f:
        f.write("Statistical Summary of Anxiety Data:\n\n")
        f.write(summary_stats.to_string())
    print("Statistical summary saved.")

    # Create visualizations
    print(create_kde_plot(df, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH))
    print(create_grouped_kde_plot(df, GROUP_COLUMN, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH))
    print(create_violin_plot(df, GROUP_COLUMN, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH))
    print(create_scatter_plot(df, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, GROUP_COLUMN, OUTPUT_PATH))

    # Prepare data for decision tree
    df_encoded = pd.get_dummies(df.copy(), columns=[GROUP_COLUMN], prefix=GROUP_COLUMN, drop_first=False)
    tree_features = [ANXIETY_PRE_COLUMN] + [col for col in df_encoded.columns if col.startswith(f"{GROUP_COLUMN}_")]
    print(create_decision_tree_plot(df_encoded, tree_features, ANXIETY_POST_COLUMN, OUTPUT_PATH))

    # Basic analysis output
    bootstrap_ci = perform_bootstrap(df[ANXIETY_POST_COLUMN], np.mean)
    print(f"Bootstrap CI for post-anxiety mean: {bootstrap_ci}")
    print(f"Analysis complete. Output saved to: {OUTPUT_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Statistical summary saved.
KDE plot created.
Grouped KDE plot created.
Violin plot created.
Scatter plot created.
Decision tree plot created.
Bootstrap CI for post-anxiety mean: ConfidenceInterval(low=2.6, high=4.4)
Analysis complete. Output saved to: /content/drive/MyDrive/output_anxiety_analysis/
